In [1]:
# Utility
import re, sys, copy, arrow, random
from helpers import *

In [6]:
@timeit
def day13():
    departure, schedule = process_lines(2020, 13)
    departure = int(departure)
    
    @timeit
    def part1():    
        buses = [int(bus) for bus in schedule.split(',') if bus != 'x']
        ts = departure - 1
        while True:
            ts = ts + 1
            for bus in buses:
                if ts % bus == 0:
                    return bus * (ts - departure)
                    
    handle_result(part1(), 2020, 13, "a")
    
    @timeit
    def part2():
        def find_depart(bus_id, offset, seed, interval):
            ts = seed
            while True:
                ts += interval
                if (ts + offset) % bus_id == 0:
                    return ts
                
        buses = [(int(b), idx) for idx, b in enumerate(schedule.split(',')) if b != 'x']
        seed = 0
        interval = 1
        for bus, offset in buses:
            ts = find_depart(bus, offset, seed, interval)
            seed = ts
            interval *= bus       
        return ts
    handle_result(part2(), 2020, 13, "b")
    
day13()

part1 0.02 ms
2305
Part a already solved with same answer: 2305
part2 0.08 ms
552612234243498
Part b already solved with same answer: 552612234243498
day13 3.91 ms


In [14]:
@timeit
def day12():
    from helpers.grids.xy import UP, LEFT, DOWN, RIGHT, chess_move
    
    def parse(line):
        return line[0], int(line[1:])
    data = process_lines(2020, 12, parse)
    
    DIRECTIONS = { 'N': UP, 'S': DOWN, 'E': RIGHT, 'W': LEFT }
    
    rotate = lambda d, x, y: (x, y) if d % 360 == 0 else rotate(d - 90, y, -x)
    
    @timeit
    def part1():
        ship, heading = origin, DIRECTIONS['E']
        for action, amount in data:
            match action:
                case 'L': heading = rotate(amount, *heading)
                case 'R': heading = rotate(-amount, *heading)
                case 'F': ship = move(ship, heading, amount)
                case   _: ship = move(ship, DIRECTIONS[action], amount)
        return cityblock_distance(origin, ship)
    handle_result(part1(), 2020, 12, "a")
        
    @timeit
    def part2():
        ship, waypoint = origin, chess_move(rights=10, ups=1)
            
        for action, amount in data:
            match action:
                case 'L': waypoint = rotate(amount, *waypoint)
                case 'R': waypoint = rotate(-amount, *waypoint)
                case 'F': ship = move(ship, waypoint, amount)
                case   _: waypoint = move(waypoint, DIRECTIONS[action], amount)
        return cityblock_distance(origin, ship)
    handle_result(part2(), 2020, 12, "b")
    
day12()

part1 1.15 ms
2458
Part a already solved with same answer: 2458
part2 1.07 ms
145117
Part b already solved with same answer: 145117
day12 9.12 ms


In [3]:
@timeit
def day11():
    from helpers.grids.rc import data_to_grid, grid_print, grid_yield, HEADINGS_ALL

    grid, hx, hy = data_to_grid(get_lines(2020, 11))
    EMPTY, OCCUPIED, FLOOR = "L", "#", "."
    
    def walk(seed, neighbors_func, occupied_limit):
        current = seed
        rv = {}
        while True:
            rv = {}
            for r, c, _ in grid_yield(current):
                pt = (r,c)
                val = current[pt]
                neighbors = [current.get(n, FLOOR) for n in neighbors_func(pt)]
                if val == EMPTY and all(n != OCCUPIED for n in neighbors):
                    val = OCCUPIED
                if val == OCCUPIED and sum(n == OCCUPIED for n in neighbors) >= occupied_limit:
                    val = EMPTY
                rv[pt] = val
            yield rv
            current = rv
            
    def is_same(g1, g2):
        for r, c, _ in grid_yield(g1):
            pt = (r,c)
            if g1[pt] != g2[pt]: return False
        return True
            
    def part1():
        prev = dict(grid)
        for state in walk(grid, neighbors8, 4):            
            if is_same(prev, state):
                return sum(state.get((r,c)) == OCCUPIED for r,c,_ in grid_yield(state))
            prev = dict(state)
    handle_result(part1(), 2020, 11, "a")
    
    def more_neighbors(point):
        for d in HEADINGS_ALL:
            tmp = point
            while True:
                tmp = add(tmp, d)
                if grid.get(tmp) is None:
                    break
                if grid[tmp] == FLOOR:
                    continue
                yield tmp
                break
    
    def part2():
        prev = dict(grid)
        for state in walk(grid, more_neighbors, 5):            
            if is_same(prev, state):
                return sum(state.get((r,c)) == OCCUPIED for r,c,_ in grid_yield(state))
            prev = dict(state)
    handle_result(part2(), 2020, 11, "b")    
            
    
day11()

2178
Part a already solved with same answer: 2178
1978
Part b already solved with same answer: 1978
day11 13357.88 ms


In [4]:
@timeit
def day10():
    adapters = [16,10,15,5,1,11,7,19,6,12,4]
    adapters = process_lines(2020, 10, int)
    
    def find_adapters(seed):
        return [adapter for adapter in adapters if adapter > seed and adapter-seed < 4]
    
    def walk():
        stats = defaultdict(int)
        current = 0
        while True:
            adapters = find_adapters(current)
            if len(adapters) == 0:
                return stats
            nxt = min(adapters)
            stats[nxt-current] += 1
            current = nxt
            
    @cache
    def options(adapters, seed):
        first, rest = adapters[0], adapters[1:]
        if first - seed > 3: return 0
        if not rest: return 1
        return options(rest, first) + options(rest, seed)
    
    def part1():
        stats = walk()
        stats[3] +=1 #final adapter to device is always +3
        return stats[1] * stats[3]
    handle_result(part1(), 2020, 10, "a")
    
    def part2():
        return options(tuple(sorted(adapters)), 0)
    handle_result(part2(), 2020, 10, "b")
            
    
day10()

2112
Part a already solved with same answer: 2112
3022415986688
Part b already solved with same answer: 3022415986688
day10 4.99 ms


In [5]:
@timeit
def day9():
    lines = process_lines(2020, 9, int)
    print(len(lines))
    
    def is_valid(corpus, check):
        for a, b in permutations(corpus, 2):
            if a+b == check: return True
        return False
    
    def find_invalid():
        for section in overlapping(lines, 26):
            corpus = section[:-1]
            current = section[-1]
            if not is_valid(corpus, current):
                yield current
    
    def part1():
        return first(find_invalid())
    handle_result(part1(), 2020, 9, "a")
    
    def part2():
        goal = first(find_invalid())
        size = 1
        while True:
            size += 1
            for section in overlapping(lines, size):
                if sum(section) == goal:
                    return min(section) + max(section)
    handle_result(part2(), 2020, 9, "b")
    
day9()

1000
70639851
Part a already solved with same answer: 70639851
8249240
Part b already solved with same answer: 8249240
day9 18.21 ms


In [6]:
@timeit
def day8():
    def parser(line):
        code, value = line.split(' ')
        return (code, int(value))

    program = process_lines(2020, 8, parser)

    def run(code):
        acc = pc = 0
        seen = set()
        order = []
        while True:
            if pc in seen or pc >= len(code):
                return (acc, pc, order, pc >= len(code))
            seen.add(pc)
            order.append(pc)
            ins, value = code[pc]
            if ins == "acc":
                acc += value
                pc +=1
            if ins == "nop":
                pc += 1
            if ins == "jmp":
                pc += value
                
    def part1():
        acc, _, _, _ = run(program)
        return acc
    handle_result(part1(), 2020, 8, "a")
    
    def part2():
        _, _, order, _ = run(program)
        changable = set(["nop", "jmp"])
        for pc in order[::-1]:
            ins, value = program[pc]
            if ins in changable:
                tmp = program[:]
                tmp[pc] = ("jmp" if ins == "nop" else "nop", value)
                acc, _, _, terminated = run(tmp)
                if terminated:
                    return acc
    handle_result(part2(), 2020, 8, "b")
        
day8()

1949
Part a already solved with same answer: 1949
2092
Part b already solved with same answer: 2092
day8 4.62 ms


In [7]:
@timeit
def day7():
    re_subrule = re.compile(r"(\d+) (.+?) bags?")

    def process(rule):
        name, subrules = rule.split(" bags contain ")
        subrules = None if "no other" in subrules else [(i[1], int(i[0])) for i in re_subrule.findall(subrules)]
        return (name, subrules)

    lines = get_lines(2020, 7)
    rules = {}
    for line in lines:
        name, bag_rules = process(line)
        rules[name] = bag_rules

    flipped = defaultdict(list)
    for key, values in rules.items():
        if values is None:
            continue
        for name, count in values:
            flipped[name].append(key)
            
    def get_options(color):
        if not flipped.get(color):
            return set()
        seen = set()
        for parent in flipped[color]:
            seen.add(parent)
            seen = seen | get_options(parent)
        return seen
    
    def get_subs(color):
        if rules[color] is None: 
            return 0
        rv = 0
        for color, qty in rules[color]:
            rv += qty + (qty * get_subs(color))
        return rv
            
    def part1():
        return len(get_options('shiny gold'))
    handle_result(part1(), 2020, 7, "a")
    
    def part2():
        return get_subs('shiny gold')
    handle_result(part2(), 2020, 7, "b")
day7()

128
Part a already solved with same answer: 128
20189
Part b already solved with same answer: 20189
day7 8.23 ms


In [8]:
@timeit
def day6():
    raw = get_raw(2020, 6)
    groups = [(g.count("\n") + 1, g.replace("\n", "")) for g in raw.split("\n\n")]
    
    group_score1 = lambda g: len(Counter(g[1]).keys())
    
    def group_score2(group):
        size, answers = group
        counter = Counter(answers)
        return sum(1 for item in counter.most_common() if item[1] == size)
        
    
    def loop(func):
        return sum(func(g) for g in groups)
    
    handle_result(loop(group_score1), 2020, 6, "a")
    handle_result(loop(group_score2), 2020, 6, "b")
            
day6()

6443
Part a already solved with same answer: 6443
3232
Part b already solved with same answer: 3232
day6 11.51 ms


In [9]:
@timeit
def day5():
    seats = get_lines(2020, 5)
    
    def bin_search(low, high, path, low_key, idx = 0):
        pivot = low + ((high-low)//2)
        d = path[idx]
        if idx + 1 == len(path):
            return low if d == low_key else high
        return bin_search(low, pivot, path, low_key, idx+1) if d == low_key else bin_search(pivot+1, high, path, low_key, idx+1)
    
    calc_coords = lambda seat: (bin_search(0, 127, seat[:7], "F"), bin_search(0, 7, seat[7:], "L"))
    calc_seat_id = lambda rc: rc[0] * 8 + rc[1]
    
    def part1():
        rv = 0
        for seat in seats:
            seat_id = calc_seat_id(calc_coords(seat))
            rv = max(rv, seat_id)
        return rv
    handle_result(part1(), 2020, 5, "a")
    
    def part2():
        ids = sorted([calc_seat_id(calc_coords(seat)) for seat in seats])
        for s1, s2 in pairwise(ids):
            if s2-s1 > 1:
                return s1+1
    handle_result(part2(), 2020, 5, "b")
    
day5()

994
Part a already solved with same answer: 994
741
Part b already solved with same answer: 741
day5 11.13 ms


In [10]:
@timeit
def day4(): 
    re_clean = re.compile(r"\n(.)", re.DOTALL)
    re_fields = re.compile(r"(byr|iyr|eyr|hgt|hcl|ecl|pid|cid):(\S+)")
    re_cid = re.compile(r"cid:\S+")
    
    raw = get_lines(2020, 4, lambda r: re_clean.sub(r" \1", r))
    
    rules = {
        "byr": { 
            "reg": re.compile(r"^\d{4}$"), 
            "valid": lambda v: 1920 <= int(v) <= 2002 
        },
        "iyr": { 
            "reg": re.compile(r"^\d{4}$"), 
            "valid": lambda v: 2010 <= int(v) <= 2020 
        },
        "eyr": { 
            "reg": re.compile(r"^\d{4}$"), 
            "valid": lambda v: 2020 <= int(v) <= 2030 
        },
        "hgt": { 
            "reg": re.compile(r"^(\d+)(cm|in)$"), 
            "valid": lambda r: (59 <= int(r[0]) <= 76) if (r[1] == "in") else (150 <= int(r[0]) <= 193) 
        },
        "hcl": { 
            "reg": re.compile(r"^#[a-f0-9]{6}$"),     
            "valid": always(True)
        },
        "ecl": { 
            "reg": re.compile(r"^(?:amb|blu|brn|gry|grn|hzl|oth)$"), 
            "valid": always(True)
        },
        "pid": { 
            "reg": re.compile(r"^\d{9}$"), 
            "valid": always(True)
        },
    }
    
    def validate(passports, secondary=None):        
        for passport in passports:
            entries = re_fields.findall(passport)
            count = len(entries)
            has_cid = re_cid.search(passport) is not None
            
            if (count == 8) or (count == 7 and has_cid is False):
                yield 1 if (secondary is None or secondary(entries) is True) else 0
            else:
                yield 0
                
    def secondary(entries):
        for code, data in entries:
            if code in rules:
                rule = rules[code]
                inp = rule["reg"].findall(data)
                if len(inp) == 0 or rule["valid"](inp[0]) is False:
                    return False
        return True
    
    def part1():
        return sum(p for p in validate(raw))
    handle_result(part1(), 2020, 4, "a")
    
    def part2():
        return sum(p for p in validate(raw, secondary))
    handle_result(part2(), 2020, 4, "b")

day4()

245
Part a already solved with same answer: 245
133
Part b already solved with same answer: 133
day4 9.40 ms


In [11]:
@timeit
def day3():
    from helpers.grids.rc import data_to_grid, chess_move
    
    grid, mc, mr = data_to_grid(get_lines(2020, 3))
    
    def traverse(movement):
        current = origin
        trees = 0
        mod = mc + 1
        
        while True:
            r, c = add(current, movement)
            if r > mr:
                return trees
            current = (r, c % mod)
            trees += 1 if grid[current] == "#" else 0
            
    mul = lambda lst: reduce(lambda x, y: x * y, lst)
    
    def part1():
        return traverse(chess_move(rights=3, downs=1))
    handle_result(part1(), 2020, 3, "a")
    
    def part2():
        moves = [
            chess_move(rights=1, downs=1),
            chess_move(rights=3, downs=1),
            chess_move(rights=5, downs=1),
            chess_move(rights=7, downs=1),
            chess_move(rights=1, downs=2),
        ]
        
        trees = [traverse(m) for m in moves]
        return mul(trees)
    handle_result(part2(), 2020, 3, "b")
        
day3()

299
Part a already solved with same answer: 299
3621285278
Part b already solved with same answer: 3621285278
day3 9.10 ms


In [12]:
@timeit
def day2():
    re_split = re.compile("[\d\w]+")
    
    def process(lines):
        for rule in lines:
            n1, n2, l, p = re_split.findall(rule)
            yield int(n1), int(n2), l, p
    
    lines = list(process(get_lines(2020, 2)))
    
    def part1():
        valid = 0
        for low, high, letter, pw in lines:
            valid += 1 if low <= pw.count(letter) <= high else 0
        return valid
    handle_result(part1(), 2020, 2, "a")
    
    def part2():
        valid = 0
        for p1, p2, letter, pw in lines:
            l1, l2 = pw[p1-1], pw[p2-1]
            valid += 1 if (l1 == letter or l2 == letter) and (l1 != l2) else 0
        return valid
    handle_result(part2(), 2020, 2, "b")
        
day2()

445
Part a already solved with same answer: 445
491
Part b already solved with same answer: 491
day2 6.22 ms


In [13]:
@timeit
def day1():
    data = map_ints(get_lines(2020, 1))
    first = second = False
    
    for a in data:
        for b in data:
            if first is False and a+b == 2020:
                first = True
                handle_result(a*b, 2020, 1, "a")
                
            for c in data:
                if second is False and a+b+c == 2020:
                    second = True
                    handle_result(a*b*c, 2020, 1, "b")
                if first and second:
                    break
day1()

138688160
Part b already solved with same answer: 138688160
445536
Part a already solved with same answer: 445536
day1 96.28 ms
